In [88]:
from pyspark.sql import SparkSession

In [93]:

spark = SparkSession.builder \
    .appName("Iceberg_Jupyter_Access") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "rest") \
    .config("spark.sql.catalog.iceberg.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.sql.defaultCatalog", "iceberg") \
    .getOrCreate()

print("Jupyter üzerinden Spark bağlantısı başarılı!")

Jupyter üzerinden Spark bağlantısı başarılı!


In [107]:
spark.sql('SHOW CATALOGS').show()

+-------------+
|      catalog|
+-------------+
|      iceberg|
|spark_catalog|
+-------------+



In [106]:
spark.sql('CREATE NAMESPACE iceberg.demo')

DataFrame[]

In [111]:
spark.sql('SHOW NAMESPACES in iceberg').show()

+---------+
|namespace|
+---------+
|     demo|
+---------+



In [119]:
spark.sql('USE iceberg')

DataFrame[]

In [121]:
spark.sql('CREATE TABLE iceberg.demo.test (id integer,name STRING)')

DataFrame[]

In [122]:
spark.sql("INSERT INTO iceberg.demo.test (id,name) values(1,'ramazan'),(2,'eren')")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".                
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [125]:
spark.sql('SELECT * FROM iceberg.demo.test where name=\'ramazan\'').show()

+---+-------+
| id|   name|
+---+-------+
|  1|ramazan|
+---+-------+



In [126]:
spark.sql('ALTER TABLE iceberg.demo.test add partition field id')

DataFrame[]

In [127]:
spark.sql('DESCRIBE TABLE iceberg.demo.test').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                  id|      int|   NULL|
|                name|   string|   NULL|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|                  id|      int|   NULL|
+--------------------+---------+-------+



In [128]:
spark.sql("INSERT INTO iceberg.demo.test (id,name) values(3,'ahmet'),(4,'mehmet')")

DataFrame[]

In [133]:
spark.sql("CALL iceberg.system.rewrite_data_files(table => 'iceberg.demo.test', strategy => 'sort',sort_order => 'id')")

DataFrame[rewritten_data_files_count: int, added_data_files_count: int, rewritten_bytes_count: bigint, failed_data_files_count: int]

In [134]:
print(spark.catalog.currentCatalog())

iceberg


In [136]:
# 1. Önce Spark'ın planına bakalım
df = spark.sql("SELECT * FROM iceberg.demo.test WHERE id = 1")
df.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('id = 1)
   +- 'UnresolvedRelation [iceberg, demo, test], [], false

== Analyzed Logical Plan ==
id: int, name: string
Project [id#709, name#710]
+- Filter (id#709 = 1)
   +- SubqueryAlias iceberg.demo.test
      +- RelationV2[id#709, name#710] iceberg.demo.test iceberg.demo.test

== Optimized Logical Plan ==
Filter (isnotnull(id#709) AND (id#709 = 1))
+- RelationV2[id#709, name#710] iceberg.demo.test

== Physical Plan ==
*(1) Filter (isnotnull(id#709) AND (id#709 = 1))
+- *(1) ColumnarToRow
   +- BatchScan iceberg.demo.test[id#709, name#710] iceberg.demo.test (branch=null) [filters=id IS NOT NULL, id = 1, groupedBy=] RuntimeFilters: []



In [137]:
spark.sql("SELECT * FROM iceberg.demo.test.history").show()

+--------------------+------------------+------------------+-------------------+
|     made_current_at|       snapshot_id|         parent_id|is_current_ancestor|
+--------------------+------------------+------------------+-------------------+
|2026-01-17 19:18:...|196083004806790204|              NULL|               true|
|2026-01-17 19:22:...|450108357846897560|196083004806790204|               true|
+--------------------+------------------+------------------+-------------------+



In [142]:
spark.sql("SELECT * FROM iceberg.demo.test FOR SYSTEM_TIME AS OF 450108357846897560").show()

+---+-------+
| id|   name|
+---+-------+
|  4| mehmet|
|  1|ramazan|
|  3|  ahmet|
|  2|   eren|
+---+-------+



In [143]:
spark.sql('DELETE FROM iceberg.demo.test where name="ramazan"')

DataFrame[]

In [144]:
spark.sql('select *  from iceberg.demo.test').show()

+---+------+
| id|  name|
+---+------+
|  2|  eren|
|  4|mehmet|
|  3| ahmet|
+---+------+



In [148]:
spark.sql("SELECT * FROM iceberg.demo.test.history").show()

+--------------------+-------------------+------------------+-------------------+
|     made_current_at|        snapshot_id|         parent_id|is_current_ancestor|
+--------------------+-------------------+------------------+-------------------+
|2026-01-17 19:18:...| 196083004806790204|              NULL|               true|
|2026-01-17 19:22:...| 450108357846897560|196083004806790204|               true|
|2026-01-17 19:29:...|3473538479800572684|450108357846897560|               true|
+--------------------+-------------------+------------------+-------------------+



In [151]:

df_eskiveri = spark.read.format("iceberg") \
    .option("as-of-time", "3473538479800572684") \
    .load("iceberg.demo.test")

df_eskiveri.show()

+---+------+
| id|  name|
+---+------+
|  4|mehmet|
|  3| ahmet|
|  2|  eren|
+---+------+



In [152]:
spark.sql("ALTER TABLE iceberg.demo.test RENAME COLUMN id TO user_id")

DataFrame[]

In [153]:
spark.sql("ALTER TABLE iceberg.demo.test ADD COLUMN status STRING")

DataFrame[]

In [154]:
spark.sql("SELECT * FROM iceberg.demo.test").show()

+-------+------+------+
|user_id|  name|status|
+-------+------+------+
|      2|  eren|  NULL|
|      4|mehmet|  NULL|
|      3| ahmet|  NULL|
+-------+------+------+



In [156]:
spark.stop()

In [160]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.utils import AnalysisException

# Iceberg Schema Evolution ayarları ile Spark'ı başlatıyoruz
spark = SparkSession.builder \
    .appName("NYC Taxi to Iceberg Optimized") \
      .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "rest") \
    .config("spark.sql.catalog.iceberg.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.sql.defaultCatalog", "iceberg") \
    .getOrCreate()

taxi_types = ['high_volume_for_hire_vehicle']
years = [str(y) for y in range(2009, 2026)]

# Veritabanını oluştur
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.nyc")

for taxi_type in taxi_types:
    print(f"\n🚀 --- Starting {taxi_type} ---")
    
    table_name = f"iceberg.nyc.{taxi_type}"
    
    for year in years:
        # Not: NYC verisi bazen ay ay da klasörlenmiş olabilir, wildcard (*) hayat kurtarır.
        raw_path = f"s3a://nyc-taxi/Dataset/{year}/{taxi_type}/*.parquet"
        
        try:
            # 1. Veriyi Oku (mergeSchema opsiyonu ile parça parça okumalarda risk azalır)
            # S3'te dosya yoksa hata vermemesi için try-except bloğu kritik.
            try:
                df = spark.read.option("mergeSchema", "true").parquet(raw_path)
            except AnalysisException:
                print(f"⚠️  No data found for {taxi_type} in {year}. Skipping...")
                continue

            if df.count() > 0:
                print(f"Processing {taxi_type} - Year: {year} ({df.count()} rows)...")

                # 2. Sütun İsimlerini Normalize Et (Hepsini küçük harfe çevir)
                # NYC verisinde VendorID ve vendor_id karmaşası çoktur.
                df = df.toDF(*[c.lower() for c in df.columns])

                # Metadata sütunlarını ekle
                df = df.withColumn("taxi_type", lit(taxi_type)) \
                       .withColumn("file_year", lit(year))

                # 3. Tablo Kontrolü ve Yazma İşlemi
                if not spark.catalog.tableExists(table_name):
                    print(f"🆕 Creating new table: {table_name}")
                    
                    # Tabloyu oluştururken Schema Evolution'a izin veriyoruz
                    df.writeTo(table_name) \
                        .tableProperty("write.format.default", "parquet") \
                        .tableProperty("write.spark.accept-any-schema", "true") \
                        .partitionedBy("file_year") \
                        .create()  # createOrReplace kullanma, yanlışlıkla silme riskin var!
                else:
                    print(f"➕ Appending to existing table: {table_name}")
                    
                    # Iceberg otomatik olarak şemayı genişletecek (yeni sütun gelirse ekler)
                    df.writeTo(table_name) \
                        .option("merge-schema", "true") \
                        .option("check-ordering", "false") \
                        .append()

                print(f"✅ Year {year} committed.")

        except Exception as e:
            print(f"❌ Error processing {taxi_type}/{year}: {str(e)}")
            # Hata olsa bile döngüyü kırma, sonraki yıla geç
            continue

    print(f"🏁 {taxi_type} processing finished.")

spark.stop()


🚀 --- Starting high_volume_for_hire_vehicle ---
⚠️  No data found for high_volume_for_hire_vehicle in 2009. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2010. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2011. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2012. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2013. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2014. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2015. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2016. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2017. Skipping...
⚠️  No data found for high_volume_for_hire_vehicle in 2018. Skipping...


26/01/17 19:49:41 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.18.0.6 executor 1): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: INT, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: INT>"
Schema that cannot be merged with the initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIME

❌ Error processing high_volume_for_hire_vehicle/2019: An error occurred while calling o2783.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 4) (172.18.0.6 executor 1): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: INT, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: INT>"
Schema that canno

26/01/17 19:49:43 WARN TaskSetManager: Lost task 1.0 in stage 1.0 (TID 6) (172.18.0.4 executor 0): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: INT, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: STRING>"
Schema that cannot be merged with the initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: T

❌ Error processing high_volume_for_hire_vehicle/2020: An error occurred while calling o2790.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 1.0 failed 4 times, most recent failure: Lost task 3.3 in stage 1.0 (TID 14) (172.18.0.6 executor 1): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: INT, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: STRING>"
Schema that c

...

✅ Year 2021 committed.


Processing high_volume_for_hire_vehicle - Year: 2022 (212416083 rows)...
➕ Appending to existing table: iceberg.nyc.high_volume_for_hire_vehicle


✅ Year 2022 committed.


26/01/17 20:00:04 WARN TaskSetManager: Lost task 0.0 in stage 22.0 (TID 288) (172.18.0.4 executor 0): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: DOUBLE, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: STRING>"
Schema that cannot be merged with the initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datet

❌ Error processing high_volume_for_hire_vehicle/2023: An error occurred while calling o2834.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 4 times, most recent failure: Lost task 0.3 in stage 22.0 (TID 294) (172.18.0.4 executor 0): org.apache.spark.SparkException: [CANNOT_MERGE_SCHEMAS] Failed merging schemas:
Initial schema:
"STRUCT<hvfhs_license_num: STRING, dispatching_base_num: STRING, originating_base_num: STRING, request_datetime: TIMESTAMP_NTZ, on_scene_datetime: TIMESTAMP_NTZ, pickup_datetime: TIMESTAMP_NTZ, dropoff_datetime: TIMESTAMP_NTZ, PULocationID: BIGINT, DOLocationID: BIGINT, trip_miles: DOUBLE, trip_time: BIGINT, base_passenger_fare: DOUBLE, tolls: DOUBLE, bcf: DOUBLE, sales_tax: DOUBLE, congestion_surcharge: DOUBLE, airport_fee: DOUBLE, tips: DOUBLE, driver_pay: DOUBLE, shared_request_flag: STRING, shared_match_flag: STRING, access_a_ride_flag: STRING, wav_request_flag: STRING, wav_match_flag: STRING>"
Schema 

In [171]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /opt/bitnami/python/lib/python3.11/site-packages/pip-23.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 468.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.2/590.2 kB 31.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [172]:
%load_ext sql

ModuleNotFoundError: No module named 'sql'

In [170]:
%sql
SELECT vendorid, COUNT(*) 
FROM iceberg.nyc.yellow_taxi 
GROUP BY 1 
LIMIT 10

SyntaxError: invalid syntax (3040630173.py, line 2)